# Coursera Capstone Project

## Battle of Neighborhoods (Week 1)


## Importing Libraries

In [19]:
# Set up import statements for all of the packages that plan to use.

import numpy as np
import pandas as pd
import random # library for random number generation
import os
import folium 
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries imported.')



Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\ahmed\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1j             |       h8ffe710_0         5.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.8 MB

The following packages will be UPDATED:

  certifi            pkgs/main::certifi-2020.12.5-py38haa9~ --> conda-forge::certifi-2020.12.5-py38haa244fe_1
  openssl              pkgs/main::openssl-1.1.1i-h2bbff1b_0 --> conda-forge::openssl-1.1.1j-h8ffe710_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2020.12.8-~ --> conda-forge::ca-certificates-2020.12.5-h5b45459_0
  conda               pkgs/main::conda-4.9.2-py

## 1. Introduction

###        Introducing the problem and the project aim 

I am a person who will move temporarily by New Cairo Compounds in Egypt and I am willing to serach for the top venues nearby such as Cafes, restaurants, shopping malls and sporting clubs.


This project is giving me the opportunity to seach out for these resources smarter and faster. Which gives me an easier methodology to reach out the proper distination whithout paying too much effort and time walking through roads searching for the ultimate destination.

The interested audience might be the peaple willing to move by the side and needs to live by certain utilities.

Limiting up my project research for the top 200 venus.


## Data Section 

###        Data desciption which are collected from the foursquare Api

The data is required to provide adequate information about the Cafes, restaurants, shopping malls and sporting clubs.

The data should help me decide where to rent my appartment with good pricing.

The data should help me make the most use out of my time with the top areas to visit and meet by.


### Data Usage 

1. Using the geolocator to locate New Cairo Address 

2. get address longitute and latitudes

3. Use the four square Api with the Client ID and Version to find out the list of venues around a radius area. 

## Getting the address and the location geographic location

In [32]:

address = 'New Cairo, Egypt'

geolocator = Nominatim(user_agent="nd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Egypt home are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Egypt home are 30.019262949999998, 31.503642836148114.


In [26]:

neighborhood_latitude=latitude
neighborhood_longitude=longitude

## Pushing the Foursquare Credentials

In [27]:

CLIENT_ID = '02CM01U2QJ0KL0REJWOXTYRC3CUEKRKABLSVE2304GPHO1DS' # your Foursquare ID


CLIENT_SECRET = 'RXYYM1GFQXST4HYZL50GAKGSLO2SU04SORWTWV2CMOCL2ELD' # your Foursquare Secret


VERSION = '20180605' # Foursquare API version

## Limiting the area and venues relative to the foursquare locations


In [28]:
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=02CM01U2QJ0KL0REJWOXTYRC3CUEKRKABLSVE2304GPHO1DS&client_secret=RXYYM1GFQXST4HYZL50GAKGSLO2SU04SORWTWV2CMOCL2ELD&v=20180605&ll=30.019262949999998,31.503642836148114&radius=5000&limit=200'

In [29]:
results = requests.get(url).json()


In [30]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
NDnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
NDnearby_venues =NDnearby_venues.loc[:, filtered_columns]

# filter the category for each row
NDnearby_venues['venue.categories'] = NDnearby_venues.apply(get_category_type, axis=1)

# clean columns
NDnearby_venues.columns = [col.split(".")[-1] for col in NDnearby_venues.columns]

NDnearby_venues.head(100)

<ipython-input-31-19ec3c907411>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  NDnearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Starbucks (ستاربكس),Coffee Shop,30.020782,31.495130
1,Point 90 Cinemas,Multiplex,30.020216,31.494624
2,Espresso Lab,Coffee Shop,30.020924,31.494965
3,Point 90,Shopping Mall,30.020339,31.494701
4,Desoky & Soda,Restaurant,30.020494,31.495544
...,...,...,...,...
95,TGI Friday's,American Restaurant,30.025980,31.495763
96,Enab Beirut (عنب بيروت),Lebanese Restaurant,30.054687,31.492358
97,Les Amis,Café,30.053018,31.488988
98,El Dahan Grill (الدهان),Kebab Restaurant,30.058860,31.489497


In [33]:

map_nd = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(NDnearby_venues['lat'], NDnearby_venues['lng'], NDnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_nd)  
    
map_nd